<a href="https://colab.research.google.com/github/IainMac32/ArtSuggesterAI/blob/main/ArtSuggesterModel2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Libraries


In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
import requests
from PIL import Image
from io import BytesIO

from PIL import Image, ImageEnhance
import random
import numpy as np

from collections import Counter
import numpy as np
from scipy.spatial import cKDTree


Mounted at /content/drive


Download Images to Google Drive (?)

Load and Preprocess Images


In [64]:

# dictionary of colour codes
colour_codes = {
    "red_normal": (255, 0, 0),
    "red_dark": (139, 0, 0),
    "red_light": (255, 102, 102),
    "green_normal": (0, 128, 0),
    "green_dark": (0, 100, 0),
    "green_light": (144, 238, 144),
    "blue_normal": (0, 0, 255),
    "blue_dark": (0, 0, 139),
    "blue_light": (173, 216, 230),
    "yellow_normal": (255, 255, 0),
    "yellow_dark": (204, 204, 0),
    "yellow_light": (255, 255, 153),
    "orange_normal": (255, 165, 0),
    "orange_dark": (255, 140, 0),
    "orange_light": (255, 200, 102),
    "purple_normal": (128, 0, 128),
    "purple_dark": (75, 0, 130),
    "purple_light": (216, 191, 216),
    "pink_normal": (255, 192, 203),
    "pink_dark": (255, 105, 180),
    "pink_light": (255, 182, 193),
    "brown_normal": (165, 42, 42),
    "brown_dark": (101, 67, 33),
    "brown_light": (222, 184, 135),
    "gray_normal": (128, 128, 128),
    "gray_dark": (105, 105, 105),
    "gray_light": (211, 211, 211),
    "cyan_normal": (0, 255, 255),
    "cyan_dark": (0, 139, 139),
    "cyan_light": (224, 255, 255),
    "magenta_normal": (255, 0, 255),
    "magenta_dark": (139, 0, 139),
    "magenta_light": (255, 153, 255),
    "lime_normal": (0, 255, 0),
    "lime_dark": (50, 205, 50),
    "lime_light": (204, 255, 204),
    "teal_normal": (0, 128, 128),
    "teal_dark": (0, 102, 102),
    "teal_light": (128, 191, 191),
    "navy_normal": (0, 0, 128),
    "navy_dark": (0, 0, 102),
    "navy_light": (173, 216, 230)
}

colour_ids = {name: idx for idx, name in enumerate(colour_codes)}

# Convert the colour_codes dict to a numpy array for faster computation
colour_codes_array = np.array(list(colour_codes.values()))

# Build a k-d tree for fast nearest neighbor search
colour_tree = cKDTree(colour_codes_array)

def rank_colours(img_array):
    colour_counter = Counter()

    # Reshape the image array for vectorized operations (128*128, 3)
    img_array = img_array.reshape(-1, 3)

    # Calculate the squared distance for each pixel to every color in the colour_codes
    distances, indices = colour_tree.query(img_array)  # Query all pixels at once

    # For each pixel, increment the count for the closest color
    for idx in indices:
        closest_colour = list(colour_codes.keys())[idx]
        colour_counter[closest_colour] += 1

    # Sort the colours by frequency (most common first)
    ranked_colour_ids = [colour_ids[colour] for colour, _ in colour_counter.most_common()]

    if len(ranked_colour_ids) < 5:
        # If fewer than 5 colours, repeat the most common colours to fill up the list
        ranked_colour_ids += [ranked_colour_ids[0]] * (5 - len(ranked_colour_ids))

    return ranked_colour_ids[:5]


In [65]:
# img_path is path to image (?)
# medium is one hot encoded: [1,0,0] = paint, [0,1,0] = pencil crayon, [0,0,1] = marker
# style is one hot encoded: [1,0,0,0,0] = abstract, [0,1,0,0,0] = realism, ...
def format_img_list(img_path, medium, style):
  try:
    # open and preprocess the original image
    img = Image.open(img_path).convert('RGB')
    img = img.resize((128, 128))
    img_array = np.array(img) # convert to NumPy and normalize

    # Reshape the 3D array (128x128x3) into 2D array (128*128)x3
    img_array = img_array.reshape(-1, 3)  # Flatten to (16384, 3)

    # find 5 most prominent colours
    colour_ranks = rank_colours(img_array)
    top_colours = np.array(colour_ranks[:5]).flatten()

    # format list of data
    # [flattened image in pixels, medium, top colours, style]
    img_flattened = img_array.flatten();
    img_data = [img_array, medium, top_colours, style]
    img_format = np.concatenate((img_flattened, medium, top_colours, style))

    return img_format

  except Exception as e:
    print(f"Error loading {img_name}: {e}")

In [66]:
image_folder = '/content/drive/My Drive/Colab Notebooks/SuggestionImages/'

categories = {'markerart': 0, 'drawings': 1, 'paintings': 2}
# abstract, art deco, expressionism, portrait, renaissance, romanticism, surrealist
styles = {'abstract': 0, 'art_deco': 1, 'expressionism': 2, 'portrait': 3, 'renaissance': 4, 'romanticism': 5, 'surrealism': 6}
data_list = []

for category, label in categories.items():
    category_path = os.path.join(image_folder, category)  # path to the medium type folder

    for style_name in os.listdir(category_path):  # for each image in the medium type folder
        style_path = os.path.join(category_path, style_name)  # construct the path to the image

        for img_name in os.listdir(style_path):  # for each image in the medium type folder
            img_path = os.path.join(style_path, img_name)  # construct the path to the image

            try:
                # medium encoding
                medium = [1, 0, 0]
                if category == 'drawings':
                  medium = [0, 1, 0]
                elif category == 'paintings':
                  medium = [0, 0, 1]

                # style encoding
                style = [0] * len(styles)
                style_index = styles.get(style_name, None)
                if style_index is not None:
                    style[style_index] = 1
                else:
                    print(f"Unrecognized style '{style_name}' in path {style_path}")
                    continue

                # format data
                img_format = format_img_list(img_path, medium, style)
                if img_format is not None:  # Only add valid processed data
                  data_list.append(img_format)
                  print(f"Processed {img_name}")


            except Exception as e:
                print(f"In Error loading {img_name}: {e}")

# Convert lists to NumPy arrays
data_np = np.array(data_list)


Processed rquiemcorpus2013jpgLargejpg.jpg
Processed reaping1911jpgLargejpg.jpg
Processed renewalsuprematistsquare1920jpgLargejpg.jpg
Processed raices2016jpgLargejpg.jpg
Processed redsgreens1969jpgLargejpg.jpg
Processed romboc1960jpgLargejpg.jpg
Processed rhythmsjpgLargejpg.jpg
Processed reflectionofabsence1990jpgLargejpg.jpg
Processed reticolofrangibile1970jpgLargejpg.jpg
Processed saluttom1979jpgLargejpg.jpg
Processed quartetojpgLargejpg.jpg
Processed redyellowbluepaintingno11974jpgLargejpg.jpg
Processed princepatutskyscommand1966jpgLargejpg.jpg
Processed savethedate20161jpgLargejpg.jpg
Processed prussianbluejpgLargejpg.jpg
Processed praguemorningblissivjpgLargejpg.jpg
Processed robainterior1972jpgLargejpg.jpg
Processed saberdance1952jpgLargejpg.jpg
Processed schwarzewelt2jpgLargejpg.jpg
Processed ronco1967jpgLargejpg.jpg
Processed prayer19131jpgLargejpg.jpg
Processed sandbank2005jpgLargejpg.jpg
Processed scatolaangoloverde1968jpgLargejpg.jpg
Processed roadcross1958jpgLargejpg.jpg
Pro

KeyboardInterrupt: 

In [82]:
data_np = np.array(data_list)

print(data_np[1][49152:])

np.savetxt('data.txt', data_np, delimiter=',', fmt='%s')


[ 0  0  1 26 23 24 25 22  1  0  0  0  0  0  0]


In [80]:
# Reverse the colour_ids mapping
reverse_colour_ids = {v: k for k, v in colour_ids.items()}

# Example: if 23 is the most common colour
colour_index = 24
colour_name = reverse_colour_ids.get(colour_index)

print(colour_name)


gray_normal


In [81]:
print(data_np)

[[214   7   7 ...   0   0   0]
 [172 171 141 ...   0   0   0]
 [255 255 255 ...   0   0   0]
 ...
 [168 167 165 ...   0   0   0]
 [ 72 101 169 ...   0   0   0]
 [237 231 219 ...   0   0   0]]



Define Model


Train Model

Test and Save Model